In [ ]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import StructType, StringType, BinaryType, IntegerType, DoubleType, TimestampType, DateType
from delta.tables import DeltaTable
from pyspark.sql.utils import AnalysisException
from pyspark.storagelevel import StorageLevel
from typing import Union, Optional

# --- Credenciais AWS ---
accessKeyId = ""
secretAccessKey = ""

# --- Sessão Spark ---
def create_spark_session() -> SparkSession:
    spark = (
        SparkSession
        .builder
        .appName("Silver Zone")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .enableHiveSupport()
        .getOrCreate()
    )
    
    spark.sparkContext.setLogLevel("WARN")

    conf = spark.sparkContext._jsc.hadoopConfiguration()
    conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
    conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
    conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    conf.set("fs.s3a.fast.upload", "true")
    conf.set("fs.s3a.bucket.all.committer.magic.enabled", "true")
    conf.set("fs.s3a.directory.marker.retention", "keep")
    conf.set("spark.driver.extraClassPath", "/usr/local/spark/jars/*")
    conf.set("spark.driver.memory", "8g")
    conf.set("spark.executor.memory", "16g")
    conf.set("fs.s3a.access.key", accessKeyId)
    conf.set("fs.s3a.secret.key", secretAccessKey)

    return spark

In [2]:
def create_tempview_from_delta(spark, delta_table_path, view_name):
    """
    Cria uma tempView a partir de uma tabela Delta.
    
    Parâmetros:
    - delta_table_path (str): Caminho para a tabela Delta
    - view_name (str): Nome da tempView a ser criada
    
    Retorno:
    - None (cria a tempView no SparkSession atual)
    """
   
    if spark is None:
        raise ValueError("Nenhuma SparkSession ativa encontrada")
    
    spark.read.format("delta").load(delta_table_path).createOrReplaceTempView(view_name)
    
    print(f"TempView '{view_name}' criada com sucesso a partir da tabela Delta em: {delta_table_path}")

In [3]:
%%time
spark = create_spark_session()

tabelas = [
    "cliente",
    "profissional",
    "servico",
    "agendamento",
    "pagamento",
    "horario_profissional",
    "promocao",
    "servico_promocao"
]

bronze = "s3a://dev-lab-02-us-east-2-bronze/db_barbearia/"
silver = "s3a://dev-lab-02-us-east-2-silver/"

CPU times: user 21.4 ms, sys: 0 ns, total: 21.4 ms
Wall time: 2.42 s


In [4]:
for t in tabelas:
    create_tempview_from_delta(spark, f"{bronze}{t}",t )

TempView 'cliente' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-bronze/db_barbearia/cliente
TempView 'profissional' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-bronze/db_barbearia/profissional
TempView 'servico' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-bronze/db_barbearia/servico
TempView 'agendamento' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-bronze/db_barbearia/agendamento
TempView 'pagamento' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-bronze/db_barbearia/pagamento
TempView 'horario_profissional' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-bronze/db_barbearia/horario_profissional
TempView 'promocao' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-east-2-bronze/db_barbearia/promocao
TempView 'servico_promocao' criada com sucesso a partir da tabela Delta em: s3a://dev-lab-02-us-

In [5]:
spark.sql("""SHOW VIEWS""").show()

+---------+--------------------+-----------+
|namespace|            viewName|isTemporary|
+---------+--------------------+-----------+
|         |         agendamento|       true|
|         |             cliente|       true|
|         |horario_profissional|       true|
|         |           pagamento|       true|
|         |        profissional|       true|
|         |            promocao|       true|
|         |             servico|       true|
|         |    servico_promocao|       true|
+---------+--------------------+-----------+



In [6]:
spark.sql("""describe cliente""").show()

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|     cliente_id|      int|   null|
|           nome|   string|   null|
|       telefone|   string|   null|
|          email|   string|   null|
|data_nascimento|     date|   null|
|  data_cadastro|timestamp|   null|
|    observacoes|   string|   null|
|          ativo|  boolean|   null|
| ingestion_time|timestamp|   null|
|         origem|   string|   null|
|  deletion_time|timestamp|   null|
+---------------+---------+-------+



### dim_cliente

In [7]:
spark.sql("""
SELECT 
  cliente_id as id,
  
  -- Dados pessoais
  nome,
  split(nome, ' ')[0] AS primeiro_nome,
  telefone,
  CASE 
    WHEN email LIKE '%@%.%' THEN lower(trim(email))
    ELSE NULL 
  END AS email,
  
  -- Informações demográficas
  data_nascimento,
  -- Faixa etária usando cálculo alternativo universal
  CASE 
    WHEN floor(months_between(current_date(), data_nascimento)/12) < 12 THEN 'Criança (<12)'
    WHEN floor(months_between(current_date(), data_nascimento)/12) BETWEEN 12 AND 17 THEN 'Adolescente (12-17)'
    WHEN floor(months_between(current_date(), data_nascimento)/12) BETWEEN 18 AND 24 THEN 'Jovem Adulto (18-24)'
    WHEN floor(months_between(current_date(), data_nascimento)/12) BETWEEN 25 AND 34 THEN 'Adulto Jovem (25-34)'
    WHEN floor(months_between(current_date(), data_nascimento)/12) BETWEEN 35 AND 44 THEN 'Adulto (35-44)'
    WHEN floor(months_between(current_date(), data_nascimento)/12) BETWEEN 45 AND 59 THEN 'Meia Idade (45-59)'
    ELSE 'Idoso (60+)'
  END AS faixa_etaria,
  
  -- Calcula idade exata (usando months_between)
  floor(months_between(current_date(), data_nascimento)/12) AS idade,
  
  -- Dia da semana de nascimento
  date_format(data_nascimento, 'EEEE') AS dia_semana_nascimento,
  
  -- Estação do ano de nascimento
  CASE 
    WHEN month(data_nascimento) BETWEEN 3 AND 5 THEN 'Primavera'
    WHEN month(data_nascimento) BETWEEN 6 AND 8 THEN 'Verão'
    WHEN month(data_nascimento) BETWEEN 9 AND 11 THEN 'Outono'
    ELSE 'Inverno'
  END AS estacao_nascimento,
  
  -- Dados de cadastro
  date(data_cadastro) AS data_cadastro,
  
  -- Tempo como cliente em dias (alternativa universal)
  datediff(current_date(), date(data_cadastro)) AS dias_como_cliente,
  
  -- Status
  ativo,
  
  -- Segmentação por tempo de cadastro
  CASE
    WHEN datediff(current_date(), date(data_cadastro)) > 365 THEN 'Cliente Antigo'
    WHEN datediff(current_date(), date(data_cadastro)) > 180 THEN 'Cliente Regular'
    ELSE 'Cliente Novo'
  END AS segmento_tempo,
  
  -- Metadados
  input_file_name() AS origem_dados,
  ingestion_time as data_carga,
  deletion_time as data_exclusao
FROM cliente
""").createOrReplaceTempView("dim_cliente")

### dim_profissional

In [8]:
spark.sql("""
    SELECT 
      profissional_id as id,
      nome,
      telefone,
      email,
      especialidade,
      data_admissao,
      months_between(current_date(), data_admissao) AS tempo_empresa_meses,
      ativo,
      input_file_name() AS origem_dados,
      ingestion_time as data_carga,
      deletion_time as data_exclusao
    FROM profissional
""").createOrReplaceTempView("dim_profissional")

### dim_servico

In [9]:
spark.sql("""
    SELECT 
      servico_id as id,
      nome,
      descricao,
      duracao_estimada,
      preco,
      CASE 
        WHEN lower(nome) LIKE '%barba%' THEN 'Barba'
        WHEN lower(nome) LIKE '%corte%' THEN 'Corte'
        WHEN lower(nome) LIKE '%pezinho%' THEN 'Pezinho'
        WHEN lower(nome) LIKE '%sobrancelha%' THEN 'Sobrancelha'
        ELSE 'Outros'
      END AS categoria_servico,
      ativo,
      input_file_name() AS origem_dados,
      ingestion_time as data_carga,
      deletion_time as data_exclusao
    FROM servico
""").createOrReplaceTempView("dim_servico")

### dim_tempo

In [10]:
df_dim_tempo = spark.sql("""
    WITH dates AS (
      SELECT explode(sequence(
        to_date('2020-01-01'), 
        to_date('2030-12-31'), 
        interval 1 day
      )) AS data_completa
    )
    SELECT 
      data_completa,
      day(data_completa) AS dia,
      month(data_completa) AS mes,
      year(data_completa) AS ano,
      quarter(data_completa) AS trimestre,
      weekofyear(data_completa) AS semana_ano,
      dayofweek(data_completa) AS dia_semana_numero,
      date_format(data_completa, 'EEEE') AS nome_dia_semana,
      (dayofweek(data_completa) IN (1, 7) OR month(data_completa) = 12 AND day(data_completa) = 25) AS feriado,
      dayofweek(data_completa) IN (1, 7) AS fim_de_semana
    FROM dates
""").createOrReplaceTempView("dim_tempo")

## dim_promocoes

### **1. Quantificação e Listagem de Serviços Inclusos**  
- **`qtd_servicos_inclusos`**: Conta quantos serviços estão associados a cada promoção.  
- **`servicos_associados`**: Lista os nomes dos serviços em uma única string (ex: *"Corte, Barba, Sobrancelha"*).  

---

### **2. Análise Temporal das Promoções**  
- **`duracao_dias_promocao`**: Calcula quantos dias a promoção ficou ativa.  
- **`dias_desde_inicio`**: Dias passados desde o início da promoção.  
- **`status_temporal`**: Classifica promoções em **"Ativa"**, **"Programada"** ou **"Encerrada"**.  
- **`mes_inicio`** e **`mes_fim`**: Nomes dos meses de início e término.  
- **`trimestre_promocao`**: Identifica se a promoção ocorreu em **Q1, Q2, Q3, Q4** ou se foi **multi-trimestral**.  

---

### **3. Categorização de Descontos e Eventos**  
- **`categoria_desconto`**: Classifica descontos como **Alto (≥30%)**, **Médio (15-29%)** ou **Baixo (<15%)**.  
- **`tipo_evento`**: Identifica promoções sazonais (ex: *"Natalina"*, *"Black Friday"*) ou regulares.  

---

### **4. Metadados e Rastreabilidade**  
- **`origem_dados`**: Registra a fonte dos dados (arquivo Parquet original).  
- **`data_carga`**: Data de ingestão dos dados para controle de atualizações.  

In [11]:
spark.sql("""
WITH promocoes AS (
    SELECT 
        promocao_id as id,
        nome,
        descricao,
        desconto_percentual,
        data_inicio,
        data_fim,
        ativo,
        deletion_time as data_exclusao,
        ingestion_time,
        input_file_name() AS origem_dados
    FROM promocao
),
servicos_por_promocao AS (
    SELECT 
        sp.promocao_id,
        COUNT(sp.servico_id) AS qtd_servicos,
        COLLECT_LIST(s.nome) AS lista_servicos
    FROM servico_promocao sp
    LEFT JOIN servico s ON sp.servico_id = s.servico_id
    GROUP BY sp.promocao_id
)

SELECT 
    p.*,
    COALESCE(sp.qtd_servicos, 0) AS qtd_servicos_inclusos,
    CONCAT_WS(', ', sp.lista_servicos) AS servicos_associados,
    datediff(p.data_fim, p.data_inicio) AS duracao_dias_promocao,
    datediff(current_date(), p.data_inicio) AS dias_desde_inicio,
    CASE 
        WHEN current_date() BETWEEN p.data_inicio AND p.data_fim THEN 'Ativa'
        WHEN current_date() < p.data_inicio THEN 'Programada'
        ELSE 'Encerrada'
    END AS status_temporal,
    CASE 
        WHEN p.desconto_percentual >= 30 THEN 'Alto desconto'
        WHEN p.desconto_percentual >= 15 THEN 'Médio desconto'
        ELSE 'Baixo desconto'
    END AS categoria_desconto,
    date_format(p.data_inicio, 'MMMM') AS mes_inicio,
    date_format(p.data_fim, 'MMMM') AS mes_fim,
    CASE 
        WHEN date_format(p.data_inicio, 'Q') = '1' AND date_format(p.data_fim, 'Q') = '1' THEN 'Q1'
        WHEN date_format(p.data_inicio, 'Q') = '2' AND date_format(p.data_fim, 'Q') = '2' THEN 'Q2'
        WHEN date_format(p.data_inicio, 'Q') = '3' AND date_format(p.data_fim, 'Q') = '3' THEN 'Q3'
        WHEN date_format(p.data_inicio, 'Q') = '4' AND date_format(p.data_fim, 'Q') = '4' THEN 'Q4'
        ELSE 'Multi-trimestral'
    END AS trimestre_promocao,
    CASE 
        WHEN lower(p.nome) LIKE '%natal%' OR (month(p.data_inicio) = 12 AND month(p.data_fim) = 12) THEN 'Natalina'
        WHEN lower(p.nome) LIKE '%dia dos pais%' OR (month(p.data_inicio) = 8 AND month(p.data_fim) = 8) THEN 'Dia dos Pais'
        WHEN lower(p.nome) LIKE '%dia das mães%' OR (month(p.data_inicio) = 5 AND month(p.data_fim) = 5) THEN 'Dia das Mães'
        WHEN lower(p.nome) LIKE '%black friday%' OR (month(p.data_inicio) = 11 AND month(p.data_fim) = 11) THEN 'Black Friday'
        ELSE 'Regular'
    END AS tipo_evento,
    p.ingestion_time as data_carga
FROM promocoes p
LEFT JOIN servicos_por_promocao sp ON p.id = sp.promocao_id
""").createOrReplaceTempView("dim_promocao")

## fato_agendamento

### **1. Informações de Clientes, Profissionais e Serviços**
- **`nome_cliente`**: Nome do cliente que fez o agendamento.
- **`nome_profissional`**: Nome do barbeiro/profissional responsável.
- **`nome_servico`**: Tipo de serviço agendado (corte, barba, etc.).
- **`preco_original`**: Preço base do serviço.
- **`duracao_estimada`**: Tempo previsto para o serviço.
- **`especialidade`**: Especialização do profissional (ex.: "Barbeiro", "Cabelereiro").

---

### **2. Dados de Promoções (Se Aplicável)**
- **`nome_promocao`**: Nome da promoção vigente.
- **`desconto_percentual`**: % de desconto aplicado (se houver promoção).
- **Filtro inteligente**: Apenas promoções válidas no período do agendamento são consideradas.

---

### **3. Enriquecimento Temporal**
- **`dia_semana_nome`**: Nome do dia da semana (ex.: "Monday").
- **`dia_semana_numero`**: Número do dia (1=Dom, 2=Seg, ..., 7=Sáb).
- **`hora_agendamento`**: Hora do agendamento (0-23).
- **`periodo_dia`**: Classificação em **"Manhã" (0-11h)**, **"Tarde" (12-17h)**, **"Noite" (18-23h)**.
- **`dias_antecedencia`**: Quantos dias antes o cliente agendou.
- **`dias_para_agendamento`**: Quanto tempo falta para o agendamento (se futuro).
- **`recencia_agendamento`**: Classificação temporal:
  - **"Últimos 30 dias"** (recente)
  - **"31-90 dias"** (intermediário)
  - **"Mais de 90 dias"** (antigo)
  - **"Futuro"** (agendamentos ainda não realizados).

---

In [20]:
spark.sql("""
    WITH promocoes_ativas AS (
      SELECT 
        sp.servico_id,
        sp.promocao_id,
        p.nome,
        p.desconto_percentual,
        p.data_inicio,
        p.data_fim
      FROM servico_promocao sp
      JOIN promocao p ON sp.promocao_id = p.promocao_id
      WHERE current_date() BETWEEN p.data_inicio AND p.data_fim
    ),
    agendamentos as (
        select 
            agendamento_id as id, 
            cliente_id,
            profissional_id,
            servico_id,
            data_hora,
            duracao,
            status,
            observacoes,
            data_criacao,
            data_atualizacao,
            ingestion_time,
            deletion_time as data_exclusao,
            input_file_name() AS origem_dados
        from agendamento
    )

    SELECT 
      a.*,
      c.nome AS nome_cliente,
      p.nome AS nome_profissional,
      s.nome AS nome_servico,
      s.preco AS preco_original,
      s.duracao_estimada,
      p.especialidade,
      pa.promocao_id,
      pa.nome AS nome_promocao,
      pa.desconto_percentual,

      date_format(a.data_hora, 'EEEE') AS dia_semana_nome,
      CASE date_format(a.data_hora, 'EEEE')
        WHEN 'Sunday' THEN 1
        WHEN 'Monday' THEN 2
        WHEN 'Tuesday' THEN 3
        WHEN 'Wednesday' THEN 4
        WHEN 'Thursday' THEN 5
        WHEN 'Friday' THEN 6
        WHEN 'Saturday' THEN 7
      END AS dia_semana_numero,

      hour(a.data_hora) AS hora_agendamento,
      CASE 
        WHEN hour(a.data_hora) < 12 THEN 'Manhã'
        WHEN hour(a.data_hora) < 18 THEN 'Tarde'
        ELSE 'Noite'
      END AS periodo_dia,

      datediff(date(a.data_hora), date(a.data_criacao)) AS dias_antecedencia,
      datediff(date(a.data_hora), current_date()) AS dias_para_agendamento,

      CASE 
        WHEN datediff(current_date(), date(a.data_hora)) BETWEEN 0 AND 30 THEN 'Últimos 30 dias'
        WHEN datediff(current_date(), date(a.data_hora)) BETWEEN 31 AND 90 THEN '31-90 dias'
        WHEN datediff(current_date(), date(a.data_hora)) > 90 THEN 'Mais de 90 dias'
        ELSE 'Futuro'
      END AS recencia_agendamento,
      a.ingestion_time as data_carga,
      True as ativo
    FROM agendamentos as a
    JOIN cliente c ON a.cliente_id = c.cliente_id
    JOIN profissional p ON a.profissional_id = p.profissional_id
    JOIN servico s ON a.servico_id = s.servico_id
    LEFT JOIN promocoes_ativas pa ON a.servico_id = pa.servico_id 
      AND date(a.data_hora) BETWEEN pa.data_inicio AND pa.data_fim
""").createOrReplaceTempView("fato_agendamento")

## fato_pagamento

### 1. **Junção de Dados Relacionados**
- Integração de **5 tabelas diferentes** (pagamento, agendamento, cliente, profissional, serviço)
- Criação de uma visão unificada do processo de pagamento
---
### 2. **Cálculos Temporais**
- `dias_entre_servico_pagamento`: Diferença em dias entre serviço e pagamento
- `categoria_tempo_pagamento`: Classificação temporal do pagamento (antecipado, no dia, etc.)
- `periodo_pagamento`: Período do dia (Manhã/Tarde/Noite)
- `dia_semana_pagamento`: Dia da semana do pagamento
---
### 3. **Indicadores Financeiros**
- `parcelamento_possivel`: Identifica se o pagamento pode ser parcelado (cartão + valor > 100)
- `diferenca_valor`: Diferença entre valor pago e preço original do serviço
---
### 4. **Classificações de Status**
- `status_consolidado`: Combina status de pagamento e agendamento para análise completa
---
### 5. **Metadados e Rastreabilidade**
- `origem_dados`: Identificação da fonte dos dados (com `input_file_name()`)
- `data_carga`: Timestamp de quando os dados foram ingeridos
---
### 6. **Melhorias de Qualidade**
- Normalização de datas (`data_pagamento_date`)
- Conversão de tipos para garantir cálculos precisos
- Nomes descritivos para todos os campos derivados

In [27]:
spark.sql("""
    WITH pagamentos as (
        select 
            pagamento_id as id,
            agendamento_id,
            valor_total,
            forma_pagamento,
            status,
            data_pagamento,
            observacoes,
            deletion_time as data_exclusao,
            input_file_name() AS origem_dados,
            ingestion_time AS data_carga
        from pagamento
    )
  SELECT 
        pg.*,
        a.data_hora AS data_agendamento,
        a.status AS status_agendamento,
        a.duracao,
        c.nome AS nome_cliente,
        p.nome AS nome_profissional,
        s.nome AS nome_servico,
        s.preco AS preco_original,
        date(pg.data_pagamento) AS data_pagamento_date,
        
        -- Correção: usando datediff (forma correta no Spark SQL)
        datediff(date(pg.data_pagamento), date(a.data_hora)) AS dias_entre_servico_pagamento,
        
        CASE 
            WHEN pg.forma_pagamento = 'cartao_credito' AND pg.valor_total > 100 THEN TRUE
            ELSE FALSE
        END AS parcelamento_possivel,
        
        -- Atualizado para usar datediff
        CASE 
            WHEN datediff(date(pg.data_pagamento), date(a.data_hora)) < 0 THEN 'Pago antecipado'
            WHEN datediff(date(pg.data_pagamento), date(a.data_hora)) = 0 THEN 'Pago no dia'
            WHEN datediff(date(pg.data_pagamento), date(a.data_hora)) BETWEEN 1 AND 7 THEN 'Pago em 1-7 dias'
            ELSE 'Pago após 7 dias'
        END AS categoria_tempo_pagamento,
        
        CASE 
            WHEN hour(pg.data_pagamento) < 12 THEN 'Manhã'
            WHEN hour(pg.data_pagamento) < 18 THEN 'Tarde'
            ELSE 'Noite'
        END AS periodo_pagamento,
        
        date_format(pg.data_pagamento, 'EEEE') AS dia_semana_pagamento,
        
        CASE 
            WHEN pg.status = 'pago' AND a.status = 'concluido' THEN 'Completo'
            WHEN pg.status = 'pendente' AND a.status = 'concluido' THEN 'Serviço concluído mas não pago'
            WHEN pg.status = 'pago' AND a.status != 'concluido' THEN 'Pago mas serviço não concluído'
            ELSE 'Outras situações'
        END AS status_consolidado,       
        True as ativo,
        -- Novo campo calculado: diferença entre valor pago e preço original
        (pg.valor_total - s.preco) AS diferenca_valor
    FROM pagamentos pg
    JOIN agendamento a ON pg.id = a.agendamento_id
    JOIN cliente c ON a.cliente_id = c.cliente_id
    JOIN profissional p ON a.profissional_id = p.profissional_id
    JOIN servico s ON a.servico_id = s.servico_id
""").createOrReplaceTempView("fato_pagamento")

## Upsert

In [14]:
def delta_table_exists(spark, path):
    """
    Verifica se uma tabela Delta existe no caminho especificado
    
    Args:
        spark: SparkSession
        path: Caminho para a tabela Delta (pode ser caminho S3, HDFS ou local)
        
    Returns:
        bool: True se a tabela existe, False caso contrário
    """
    try:
        DeltaTable.forPath(spark, path)
        return True
    except AnalysisException as e:
        if 'is not a Delta table' in str(e) or 'Path does not exist' in str(e):
            return False
        raise
    except Exception as e:
        # Captura outros possíveis erros
        if 'does not exist' in str(e):
            return False
        raise 

In [15]:
def upsert_with_delete_track(spark: SparkSession, delta_path: str, pk_column: str, ingestion_time_column: str = "data_carga", table_name: str = None):
    delta_table = DeltaTable.forPath(spark, delta_path)
    target_df = delta_table.toDF()
    
    source_df= spark.sql(f"select * from {t}")
    source_df.createOrReplaceTempView("source_data")
    target_df.createOrReplaceTempView("target_data")
    
    
    records_to_deactivate = spark.sql(f"""
        SELECT t.{pk_column}
        FROM target_data t
        LEFT JOIN {table_name} s ON t.{pk_column} = s.{pk_column}
        WHERE s.{pk_column} IS NULL AND t.ativo = true
    """)

    count_to_deactivate = records_to_deactivate.count()
    print(f"Registros a desativar: {count_to_deactivate}")
    records_to_deactivate.show(truncate=False)
    ids_to_deactivate = records_to_deactivate.select(pk_column).rdd.flatMap(lambda x: x).collect()

    if ids_to_deactivate:
        print(f"Encontrados {len(ids_to_deactivate)} registros para desativar")

        delta_table.update(
            condition=F.col(pk_column).isin(ids_to_deactivate) & (F.col("ativo") == True),
            set={
                "ativo": F.lit(False),
                "data_exclusao": F.current_timestamp(),
                "data_carga": F.current_timestamp()
            }
        )
        print("Registros desativados.")
    else:
        print("Nenhum registro para desativar encontrado.")
    
    # UPSERT
    delta_table.alias("target").merge(
        source_df.alias("source"),
        f"target.{pk_column} = source.{pk_column}"
    ).whenMatchedUpdate(
        condition=f"source.{ingestion_time_column} > target.{ingestion_time_column}",
        set={
            col: f"source.{col}"
            for col in source_df.columns
            if col != pk_column and col in target_df.columns
        }
    ).whenNotMatchedInsertAll().execute()

    print("UPSERT concluído!\n")

In [33]:
tabelas_silver = [
    "dim_cliente",
    "dim_profissional",
    "dim_servico",
    "dim_tempo",
    "fato_agendamento",
    "fato_pagamento",
    "dim_promocao"
]

In [34]:
%%time
contador=0
for t in tabelas_silver:
    contador+=1
    print(f"{contador} / {len(tabelas_silver)} - Inicando ingestão na bronze para a tabela de {t}")
    if delta_table_exists(spark, f"{silver}{t}") and t != 'dim_tempo':
        upsert_with_delete_track(
            spark,
            delta_path=f"{silver}{t}/",
            pk_column='id',
            ingestion_time_column="data_carga",
            table_name=t
        )
    else:
        df_silver = spark.sql(f"select * from {t}")
        df_silver.write.mode("overwrite").format("delta").save(f"{silver}{t}/")
        

1 / 7 - Inicando ingestão na bronze para a tabela de dim_cliente
Registros a desativar: 0
+---+
|id |
+---+
+---+

Nenhum registro para desativar encontrado.
UPSERT concluído!

2 / 7 - Inicando ingestão na bronze para a tabela de dim_profissional
Registros a desativar: 0
+---+
|id |
+---+
+---+

Nenhum registro para desativar encontrado.
UPSERT concluído!

3 / 7 - Inicando ingestão na bronze para a tabela de dim_servico
Registros a desativar: 0
+---+
|id |
+---+
+---+

Nenhum registro para desativar encontrado.
UPSERT concluído!

4 / 7 - Inicando ingestão na bronze para a tabela de dim_tempo
5 / 7 - Inicando ingestão na bronze para a tabela de fato_agendamento
Registros a desativar: 0
+---+
|id |
+---+
+---+

Nenhum registro para desativar encontrado.
UPSERT concluído!

6 / 7 - Inicando ingestão na bronze para a tabela de fato_pagamento
Registros a desativar: 0
+---+
|id |
+---+
+---+

Nenhum registro para desativar encontrado.
UPSERT concluído!

7 / 7 - Inicando ingestão na bronze par

In [35]:
#validação duplicidade
for t in tabelas_silver:
    if t != 'dim_tempo':
        print(t)
        spark.sql(f"""
            select 
                '{t}' as tabela,
                id,
                count(1) as qtd
            from {t}
            group by 1,2
            having count(1) > 1
        """).show()

dim_cliente
+------+---+---+
|tabela| id|qtd|
+------+---+---+
+------+---+---+

dim_profissional
+------+---+---+
|tabela| id|qtd|
+------+---+---+
+------+---+---+

dim_servico
+------+---+---+
|tabela| id|qtd|
+------+---+---+
+------+---+---+

fato_agendamento
+------+---+---+
|tabela| id|qtd|
+------+---+---+
+------+---+---+

fato_pagamento
+------+---+---+
|tabela| id|qtd|
+------+---+---+
+------+---+---+

dim_promocao
+------+---+---+
|tabela| id|qtd|
+------+---+---+
+------+---+---+

